In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 6.5 MB/s 
     |████████████████████████████████| 1.6 MB 29.0 MB/s 
     |████████████████████████████████| 9.3 MB 41.4 MB/s 
     |████████████████████████████████| 108 kB 56.1 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [7]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [8]:
df.drop(['index'],axis=1,inplace=True)

In [9]:
df_5percent=df[0:3386] 

In [10]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.357401,0.362120,96256,7.621281,1,1
1,144,3,4,65535,184,272,8,1,0,1,...,1,3,5,1,5.861511,4.873827,787048,5.864200,1,0
2,144,3,4,65535,184,256,5,1,0,1,...,1,1,4,0,6.543478,4.141282,187216,6.394658,1,0
3,144,3,4,65535,184,192,2,1,0,1,...,1,1,1,0,0.000000,0.000000,18920,5.613612,1,0
4,144,3,4,65535,184,248,5,1,0,1,...,1,1,4,0,6.284010,4.139544,339968,6.436206,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,80,2,4,65535,184,256,8,1,0,1,...,1,4,4,0,0.000000,0.000000,331264,7.322217,0,1
3382,144,2,4,65535,184,192,6,1,1,1,...,1,0,6,0,5.600852,2.860561,79360,6.302757,1,1
3383,144,3,4,65535,184,248,7,1,0,1,...,1,3,4,0,6.376263,2.426194,150736,6.576280,1,0
3384,144,3,4,65535,184,208,4,1,1,1,...,1,0,4,0,6.650924,3.978705,141824,7.475985,0,1


In [11]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (1646, 69)
class 1: (1740, 69)


In [12]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [13]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     232.0               5.0   
1      144.0   3.0        4.0     65535.0  184.0     272.0               8.0   
2      144.0   3.0        4.0     65535.0  184.0     256.0               5.0   
3      144.0   3.0        4.0     65535.0  184.0     192.0               2.0   
4      144.0   3.0        4.0     65535.0  184.0     248.0               5.0   
...      ...   ...        ...         ...    ...       ...               ...   
3381    80.0   2.0        4.0     65535.0  184.0     256.0               8.0   
3382   144.0   2.0        4.0     65535.0  184.0     192.0               6.0   
3383   144.0   3.0        4.0     65535.0  184.0     248.0               7.0   
3384   144.0   3.0        4.0     65535.0  184.0     208.0               4.0   
3385   144.0   3.0        4.0     65535.0  184.0     216.0               4.0   

      C

In [14]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,3386.000000,3386.000000,3386.00000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.0,...,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3386.000000,3.386000e+03,3386.000000,3386.000000,3386.000000
mean,149.152983,14.751034,11.55759,65111.859716,212.828706,222.705257,4.668340,0.991140,0.344359,1.0,...,0.998819,1.354105,3.314235,0.151802,4.971194,2.538086,8.380776e+05,6.351636,0.546663,0.513881
std,428.132745,487.145637,312.18382,5150.501378,1201.389447,48.373009,1.893981,0.093724,0.475229,0.0,...,0.034355,1.589332,1.152949,0.358881,2.498890,2.688642,6.002842e+06,1.102873,0.497891,0.499881
min,0.000000,0.000000,0.00000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.942085,0.000000,0.000000
25%,144.000000,3.000000,4.00000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,4.328078,0.000000,6.092800e+04,5.700839,0.000000,0.000000
50%,144.000000,3.000000,4.00000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.146370,1.581760,1.208320e+05,6.374219,1.000000,1.000000
75%,144.000000,3.000000,4.00000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.505469,4.707907,2.914120e+05,7.245522,1.000000,1.000000
max,17739.000000,20050.000000,12851.00000,65535.000000,65534.000000,648.000000,29.000000,1.000000,1.000000,1.0,...,1.000000,26.000000,8.000000,1.000000,7.999859,7.999620,1.657081e+08,7.999997,1.000000,1.000000


In [15]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [16]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [17]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [18]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      52.5
0.0      47.5
dtype: float64

In [19]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      65.0
0.0      35.0
dtype: float64

In [20]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [21]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [22]:
circuit(weights_init, data[0])

tensor(0.09075213, requires_grad=True)

In [23]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [24]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [25]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [26]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [27]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.35842219, requires_grad=False), tensor(-0.16926717, requires_grad=False)], Y = -1
X = [tensor(-0.29719906, requires_grad=False), tensor(-0.21244711, requires_grad=False)], Y =  1
X = [tensor(-0.29878411, requires_grad=False), tensor(-0.14295891, requires_grad=False)], Y =  1
X = [tensor(-0.2970811, requires_grad=False), tensor(-0.21879354, requires_grad=False)], Y = -1
X = [tensor(-0.2990181, requires_grad=False), tensor(-0.1433886, requires_grad=False)], Y = -1


In [28]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [29]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.0866648 | Accuracy: 0.5312500 
Iter:     2 | Cost: 1.1783277 | Accuracy: 0.5312500 
New best
Iter:     3 | Cost: 0.9892796 | Accuracy: 0.5500000 
Iter:     4 | Cost: 0.9250287 | Accuracy: 0.5500000 
Iter:     5 | Cost: 0.9324894 | Accuracy: 0.5375000 
New best
Iter:     6 | Cost: 0.9104010 | Accuracy: 0.5812500 
Iter:     7 | Cost: 0.9011857 | Accuracy: 0.5500000 
Iter:     8 | Cost: 0.9457131 | Accuracy: 0.5562500 
Iter:     9 | Cost: 0.9490797 | Accuracy: 0.5562500 
Iter:    10 | Cost: 1.0301616 | Accuracy: 0.5625000 
Iter:    11 | Cost: 1.0168123 | Accuracy: 0.5687500 
Iter:    12 | Cost: 0.9031563 | Accuracy: 0.5625000 
Iter:    13 | Cost: 0.9872482 | Accuracy: 0.5250000 
Iter:    14 | Cost: 1.1779889 | Accuracy: 0.5312500 
Iter:    15 | Cost: 1.3115960 | Accuracy: 0.5312500 
Iter:    16 | Cost: 1.2577278 | Accuracy: 0.5312500 
Iter:    17 | Cost: 1.0885003 | Accuracy: 0.5312500 
Iter:    18 | Cost: 0.9565003 | Accuracy: 0.5187500 
Iter:    19 | Cost:

In [30]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [31]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.9775440802489987, Accuracy: 56.99999999999999%


In [32]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,1.0,1.0
2,1.0,-1.0
3,1.0,-1.0
4,1.0,1.0
5,1.0,1.0
6,-1.0,1.0
7,1.0,1.0
8,1.0,-1.0
9,1.0,-1.0


In [33]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         3
         1.0       0.88      0.62      0.73        37

    accuracy                           0.57        40
   macro avg       0.44      0.31      0.37        40
weighted avg       0.82      0.57      0.68        40

0.8846153846153846
0.6216216216216216
0.7301587301587302
0.3108108108108108
